In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving tweets_flagged_v2.csv to tweets_flagged_v2.csv


In [ ]:
import pandas as pd
data_df = pd.read_csv("tweets_flagged_v2.csv")
data_df = data_df.drop(columns = ["Unnamed: 0"])

In [ ]:
data_df["Toxicity"].value_counts()

,count
Toxicity,
0,32592
1,24153


In [ ]:
data_df.head(5)

,Toxicity,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [ ]:
data_df.shape

(56745, 2)

In [ ]:
data_df["Toxicity"].value_counts()

,count
Toxicity,
0,32592
1,24153


In [ ]:
for _ in range(10):
    random_ind = np.random.randint(0, len(data_df))
    random_data = data_df.iloc[random_ind]
    print(random_data["tweet"], random_data["Toxicity"])

I wonder who wrote that blog... Either a hoe dat let a few of them niggas beat to no avail, or a bitch nigga who hoe let a few of em beat 1
critics threaten boycotts of simon &amp; schuster over milo yiannopoulos book deal | the huffington post   1
look who can work and travel!!! this guy right here!   #teamo @user  0
Ha bitch 1
"if she can suck your dick with a dip in, she's not only a keeper but a true redneck woman." - Kim's Dad. 1
 @user i'll keep my 2yo &amp; 5yo clear of nebulous leftisms like "diversity" &amp; "inclusiveness" until their crap filter's good. hâ¦ 0
i'm lien cuz i'm not getting up   0
@user todays date is 16-06-16. at 16:00 hours would be good for #spiritshapers announcement looks fancy isn't   #waiting #gs16 0
The tweeter gurl is on Tumblr using a taser on roaches that I'm assuming are in her house. You bitches have no shame. 1
RT @JustReIatabIe: when you die on flappy bird &#128514; http://t.co/Pu7uqxJFMc 1


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
X = data_df["tweet"].values
y = data_df["Toxicity"].values

In [ ]:
sequences = [sequence for sequence in X]
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [ ]:
model_inputs

{'input_ids': <tf.Tensor: shape=(56745, 481), dtype=int32, numpy=
array([[  101,  1030,  5310, ...,     0,     0,     0],
       [  101,  1030,  5310, ...,     0,     0,     0],
       [  101, 12170, 14945, ...,     0,     0,     0],
       ...,
       [  101,  2402, 10131, ...,     0,     0,     0],
       [  101,  2017,  2226, ...,     0,     0,     0],
       [  101,  1066,  1066, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(56745, 481), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(56745, 481), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype

In [ ]:
import tensorflow as tf


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((model_inputs['input_ids'],y))

In [ ]:
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [ ]:
# training pipeline
train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

In [ ]:
model = Sequential(name="text-classifier")
model.add(Embedding(len(tokenizer.get_vocab()), 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))

model.add(Dense(128, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))

model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.summary()

Model: "text-classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss = "binary_crossentropy", optimizer = "Adam")

In [ ]:
with tf.device("/device:GPU:0"):  # ensures GPU usage if available
    history = model.fit(
        train,                   # your training data
        epochs=1,                # number of passes through dataset
        batch_size=16,           # how many samples per gradient update
        validation_data=val      # validation data tuple
    )

2482/2482 ━━━━━━━━━━━━━━━━━━━━ 862s 345ms/step - loss: 0.2753 - val_loss: 0.1380


In [44]:
from tensorflow.keras.metrics import Precision, Recall, Accuracy
import numpy as np

# Initialize metrics
pre = Precision()
rec = Recall()
acc = Accuracy()

# Iterate over the test dataset
for batch in test.as_numpy_iterator():
    x_true, y_true = batch

    yHat = model.predict(x_true)

    # Update metrics
    pre.update_state(y_true, y_hat)
    rec.update_state(y_true, y_hat)
    acc.update_state(y_true, y_hat)

# Print final results
print("Precision:", pre.result().numpy())
print("Recall:", rec.result().numpy())
print("Accuracy:", acc.result().numpy())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━